In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os


In [ ]:

# === Konversi RGB ke HSV manual ===
def convert_rgb_to_hsv(r, g, b):
    r, g, b = [x / 255.0 for x in (r, g, b)]
    max_c, min_c = max(r, g, b), min(r, g, b)
    delta = max_c - min_c

    if delta == 0:
        h = 0
    elif max_c == r:
        h = (60 * ((g - b) / delta)) % 360
    elif max_c == g:
        h = (60 * ((b - r) / delta)) + 120
    else:
        h = (60 * ((r - g) / delta)) + 240

    s = 0 if max_c == 0 else delta / max_c
    v = max_c

    return int(h / 2), int(s * 255), int(v * 255)

# === Hitung histogram HSV dari gambar ===
def calculate_hsv_histogram(image_path, bins=(30, 32, 32)):
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Gambar tidak ditemukan: {image_path}")

    h_bin, s_bin, v_bin = bins
    h_hist, s_hist, v_hist = np.zeros(h_bin), np.zeros(s_bin), np.zeros(v_bin)

    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            b, g, r = image[y, x]
            h, s, v = convert_rgb_to_hsv(r, g, b)
            h_hist[min(int(h / (180 / h_bin)), h_bin - 1)] += 1
            s_hist[min(int(s / (256 / s_bin)), s_bin - 1)] += 1
            v_hist[min(int(v / (256 / v_bin)), v_bin - 1)] += 1

    # Normalisasi
    for hist in (h_hist, s_hist, v_hist):
        max_val = np.max(hist)
        if max_val > 0:
            hist /= max_val

    return np.concatenate([h_hist, s_hist, v_hist])

# === Utility untuk pelatihan dan klasifikasi ===
def average_histogram_minimum(histograms):
    return np.min(np.stack(histograms), axis=0)

def extract_feature_vector(histogram, base_histogram):
    return np.abs(histogram - base_histogram)

def nearest_class_index(test_feature, reference_features):
    distances = [np.mean(np.abs(test_feature - ref)) for ref in reference_features]
    return int(np.argmin(distances))

# === Simpan histogram sebagai grafik ===
def save_histogram_plot(hist, title, output_name):
    plt.figure(figsize=(10, 4))
    plt.plot(hist, label=title)
    plt.title(title)
    plt.grid(True)
    plt.tight_layout()
    os.makedirs("output", exist_ok=True)
    plt.savefig(f"output/{output_name}.png")
    plt.close()

# === Proses pelatihan data ===
data_paths = {
    "Hijau": "dataset_tomat/hijau.bmp",
    "Campur": "dataset_tomat/campur.bmp",
    "Merah": "dataset_tomat/merah.bmp"
}

hist_data = {label: calculate_hsv_histogram(path) for label, path in data_paths.items()}
general_hist = average_histogram_minimum(list(hist_data.values()))
feature_vectors = [extract_feature_vector(h, general_hist) for h in hist_data.values()]

# Simpan grafik histogram
for label, hist in hist_data.items():
    save_histogram_plot(hist, f"Histogram Spesifik Tomat {label}", f"hist_tomat_{label.lower()}")

save_histogram_plot(general_hist, "Histogram Umum HSV", "hist_general")

# === Pengujian klasifikasi ===
uji_path = "dataset_tomat/uji-merah.bmp"
# uji_path = "dataset_tomat/uji-hijau.bmp"
# uji_path = "dataset_tomat/uji-campur.bmp"
uji_image = cv2.imread(uji_path)
if uji_image is None:
    raise FileNotFoundError(f"Gambar uji tidak ditemukan: {uji_path}")

uji_hist = calculate_hsv_histogram(uji_path)
uji_feature = extract_feature_vector(uji_hist, general_hist)
predicted_index = nearest_class_index(uji_feature, feature_vectors)
predicted_label = list(hist_data.keys())[predicted_index]

print(f"Hasil klasifikasi: {predicted_label}")

# Tampilkan dan simpan hasil klasifikasi
cv2.putText(uji_image, predicted_label, (40, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
os.makedirs("hasil", exist_ok=True)
cv2.imwrite("hasil/klasifikasi_uji_merah.png", uji_image)
cv2.imshow("Klasifikasi", uji_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Hasil klasifikasi: Merah
